In [13]:
import pandas as pd
import requests
import time

In [14]:
API_KEY = 'YOUR_API_KEY'

In [15]:
route_info = """
1. ISBT Maharana Pratap Bus (T)
2. ISBT Ring Road
3. Metro Train Depot
4. Shyam Giri Mandir
5. Shastri Park
6. Shastri Park
7. Dharampura
8. Seelampur Metro Station
9. Seelam Pur Petrol Pump
10. Welcome Metro Station
11. Kanti Nagar Extension
12. Jharkhandi
13. East Azad Nagar
14. Swarn Cinema
15. Krishna Nagar A Block
16. Hans Apartment
17. Arjun Nagar
18. Radheypuri
19. Jagat Puri A-Block
20. Jagatpuri F-Block
21. Hasan Pur Depot (Patparganj Depot)
22. Hassanpur Village
23. Gazipur Depot
24. Anand Vihar Terminal
"""

In [19]:
stops_list = [line.split('. ', 1)[1].strip() for line in route_info.strip().split('\n') if line.strip()]


In [22]:


def get_coordinates(stop,city = 'Delhi'):
    url = f'https://api.tomtom.com/search/2/geocode/{stop}, {city}.json'
    params = {
     'key':API_KEY
    }
    max_retry = 3
    for attempt in range(max_retry):
        try:
            response = requests.get(url,params = params,timeout = 10)
            response.raise_for_status()
            data = response.json()
            if data['results']:
                latitude = data['results'][0]['position']['lat']
                longitude = data['results'][0]['position']['lon']
                return latitude,longitude
            else:
                print(f"No reults found for {stop}")
                return None,None
        except requests.exception.RequestException as e:
             if attempt < max_retry - 1:
                 print(f"Error occoured: {e}. Retrying in 5 seconds...")
                 time.sleep(5)
             else   :
                 print(f"failed to get coordinates for {stop} after {max_retry} attempt")
                 return None,None             


In [24]:
coordinates = {}

for stop in stops_list:
    lat,lon = get_coordinates(stop)
    coordinates[stop] = {'latitude':lat,'longitude':lon}
    time.sleep(1)

No reults found forSeelam Pur Petrol Pump


In [37]:
def get_traffic(latitude,longitude):
    url = f'https://api.tomtom.com/traffic/services/4/flowSegmentData/absolute/10/json'
    
    params = {
        'point': f'{latitude},{longitude}',
        'key': API_KEY
    }
    response = requests.get(url,params = params)
    if response.status_code  == 200:
        data = response.json()
        flow_data = {
            'Current Speed(km/h)' : data['flowSegmentData']['currentSpeed'],
            'Free Flow Speed(km/h)' : data['flowSegmentData']['freeFlowSpeed'],
            'Confidence' : data['flowSegmentData']['confidence'],
            'Current Travel Time(s)' : data['flowSegmentData']['currentTravelTime'],
            'Free Flow Time(s)' : data['flowSegmentData']['freeFlowTravelTime'],
            'Coordinates' : data['flowSegmentData']['coordinates']['coordinate']
        }
        return flow_data
    else:
        print(f"Error: Unable to fetch traffic data. Status code: {response.status_code}")
        return None   
          
    

In [38]:
def get_routes(start_lat,start_lon,end_lat,end_lon):
    url = f'https://api.tomtom.com/routing/1/calculateRoute/{start_lat},{start_lon}:{end_lat},{end_lon}/json'
    params = {
    'traffic': 'true',
    'key': API_KEY
     }
    response = requests.get(url,params=params)
    if response.status_code  == 200:
        data = response.json()
        summary = data['routes'][0]['summary']
        route_data = {
            'Distance (km)' :summary['lengthInMeters']/1000,
            'Travel Time (Min)' : summary['travelTimeInSeconds']/60,
            'Traffic Delay (Min)': summary['trafficDelayInSeconds']/60
            }
        return route_data
    else:
        print(f"Error: Unable to fetch route data. Status code: {response.status_code}")
        return None 


In [39]:
all_flow_data = []
all_route_data = []

for location,coord in coordinates.items():
    flow_data = get_traffic(coord['latitude'],coord['longitude'])
    if flow_data:
        flow_data['Location'] = location
        all_flow_data.append(flow_data)
    time.sleep(1)    

Error: Unable to fetch traffic data. Status code: 400


In [45]:
location = list(coordinates.keys())
for i in range(len(location) - 1):
    start = coordinates[location[i]]
    end = coordinates[location[i+1]]
    route_data = get_routes(start['latitude'],start['longitude'],end['latitude'],end['longitude'])
    if route_data:
        route_data['From'] = location[i]
        route_data['To'] = location[i+1]
        all_route_data.append(route_data)
    time.sleep(1)    

Error: Unable to fetch route data. Status code: 400
Error: Unable to fetch route data. Status code: 400
Error: Unable to fetch route data. Status code: 400
Error: Unable to fetch route data. Status code: 400


In [47]:
df_flow = pd.DataFrame(all_flow_data)
df_route = pd.DataFrame(all_route_data)

In [49]:
df_flow.to_csv('Flow.csv',index = False)
df_route.to_csv('Route.csv',index = False)

In [56]:
df_combined = pd.merge(df_flow,df_route,left_on = 'Location',right_on = 'From',how = 'outer')
df_combined = df_combined.dropna()
df_combined.drop(df_combined.tail(1).index,inplace=True) 
df_combined.to_csv('FinalData.csv',index = False)
